# Algorítmo de resolução do problema proposto sem restrições adicionais

Biblioteca de otimização usada: cvxpy

# Instalação da biblioteca

In [9]:
%pip install cvxpy

     -------------------------------------- 885.1/885.1 kB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 8.2/8.2 MB 4.8 MB/s eta 0:00:00
     -------------------------------------- 292.5/292.5 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 72.0/72.0 kB 4.1 MB/s eta 0:00:00
     -------------------------------------- 83.2/83.2 kB 775.7 kB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.0
[notice] To update, run: C:\Users\maxna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Declaração da classse de dados

In [14]:
# Classe de dados

from dataclasses import dataclass

@dataclass
class ValuableItem:
    opcao: str
    value: float
    retorno_esperado: float

    @property
    def value_razao(self) -> float:
        "Returns retorno esperado / value"
        return self.retorno_esperado / (self.value + 1e-9)

### Função de montagem da tabela

In [15]:
import pandas as pd 
from typing import List

def items_to_table(opcao: List[ValuableItem]) -> pd.DataFrame:
  records = [{
          'Opção': i.opcao,
          'Valor ($)': i.value,
          'Retorno esperado ($)': i.retorno_esperado
  } for i in opcao]
  records.append({
    'Opcao': 'Total',
    'Valor ($)': sum(i.value for i in opcao),
    'Retorno esperado ($)': sum(i.retorno_esperado for i in opcao)
  })
  return pd.DataFrame.from_records(records)

### Declaração dos dados de entrada

In [17]:
capacity = 1000000
valores = [
  470006,400000,176000,270000,340000,230000,50000,440000] #pesos

retorno_esperado = [
  410000,330000,140000,250000,326000,326000,90000,190006] # utilidade

available_items = [ValuableItem(f'opcao {i+1}', v, w) for i, (v, w) in enumerate(zip(valores, retorno_esperado))] 

items_to_table(available_items)

,Opção,Valor ($),Retorno esperado ($),Opcao
0,opcao 1,470006,410000,NaN
1,opcao 2,400000,330000,NaN
2,opcao 3,176000,140000,NaN
3,opcao 4,270000,250000,NaN
4,opcao 5,340000,326000,NaN
5,opcao 6,230000,326000,NaN
6,opcao 7,50000,90000,NaN
7,opcao 8,440000,190006,NaN
8,NaN,2376006,2062006,Total


# Algorítmo antigo

In [8]:
import cvxpy as cp

def knapsack(values, weights, capacity):
    x = cp.Variable(len(values), boolean=True)
    objective = cp.Maximize(values * x)
    constraints = [weights * x <= capacity,
                   x >= 0,
                   x <= 1]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return prob.value, x.value

values = retorno_esperado
weights = valores
capacity = capacity

optimal_value, selected_items = knapsack(values, weights, capacity)
print(f'Selected items: {selected_items}')
print(f'Optimal value: {optimal_value}')

Selected items: [0. 1. 0. 1. 0. 1. 1. 0.]
Optimal value: 996000.0


C:\Users\maxna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 3 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\maxna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matri

# Algoritmo novo com classes

In [18]:
import cvxpy as cp

class KnapsackProblem:
    def __init__(self, values, weights, capacity):
        self.values = values
        self.weights = weights
        self.capacity = capacity
        self.x = cp.Variable(len(values), boolean=True)
        self.objective = cp.Maximize(cp.sum(cp.multiply(values, self.x)))
        self.constraints = [cp.sum(cp.multiply(weights, self.x)) <= capacity,
                            self.x >= 0, self.x <= 1]
        self.prob = cp.Problem(self.objective, self.constraints)

    def solve(self):
        result = self.prob.solve()
        return result, [int(val) for val in self.x.value]

# Define os valores, pesos e capacidade da mochila
values = retorno_esperado
weights = valores
capacity = capacity

# Cria uma instância da classe KnapsackProblem
knapsack_problem = KnapsackProblem(values, weights, capacity)

# Resolve o problema da mochila
result, selected_items = knapsack_problem.solve()

# Imprime a solução
print("Solução ótima:", result)
print("Itens selecionados:", selected_items)

Solução ótima: 996000.0
Itens selecionados: [0, 1, 0, 1, 0, 1, 1, 0]
